In [1]:
#Entrar a cada link en archive y sacar de cada link los links relacionados para info adicional , 1 dataframe de fecha con link 
#2.dataframe link con links
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.request
from urllib.request import urlopen

In [12]:
#Url principal a scrapp archive
archive='https://apod.nasa.gov/apod/archivepix.html'

#Request + soup
html=requests.get(archive).content
soup=BeautifulSoup(html,'lxml')
soup


<!DOCTYPE html>
<html>
<head>
<title> Astronomy Picture of the Day Archive 2015
</title>
<!-- gsfc meta tags -->
<meta content="661" name="orgcode"/>
<meta content="phillip.a.newman" name="rno"/>
<meta content="Jerry.T.Bonnell.1" name="content-owner"/>
<meta content="Stephen.F.Fantasia.1" name="webmaster"/>
<meta content="astronomy picture of the day dated archive listing" name="description"/>
<!-- -->
<script id="_fed_an_ua_tag" src="//dap.digitalgov.gov/Universal-Federated-Analytics-Min.js?agency=NASA">
</script>
</head>
<body alink="#FF0000" bgcolor="#FFF5FF" link="#0000FF" text="#000000" vlink="#7F0F9F">
<center>
<table>
<tr>
<td> <img alt="logo" src="apod.gif"/> </td>
<td> <h1> Astronomy Picture of the Day Archive</h1> </td>
</tr>
</table>
</center>
<center>
<hr/>
| 
<a href="lib/aptree.html">Index</a> |
<a href="http://antwrp.gsfc.nasa.gov/cgi-bin/apod/apod_search">Search</a> |
<a href="astropix.html">Today's Picture</a> |
<br/>
<hr/>
<b>This archive list links to previous daily 

In [16]:

archive_extract=[b for b in soup.findAll('b')]
archive_extract
# names2=[a.string for a in soup.findAll('a')]
# names2

[<b>This archive list links to previous daily APOD pages
 from the current date through January 1, 2015.
 <br/>An archive of all
 existing APOD pages (current date through June 16, 1995) can be
 found here: <a href="archivepixFull.html">APOD Full Archive</a>.</b>, <b>
 2020 May 13:  <a href="ap200513.html">Jupiter in Infrared from Gemini</a><br/>
 2020 May 12:  <a href="ap200512.html">Lyrid Meteors from the Constellation Lyra</a><br/>
 2020 May 11:  <a href="ap200511.html">Behind Betelgeuse</a><br/>
 2020 May 10:  <a href="ap200510.html">The Porpoise Galaxy from Hubble</a><br/>
 2020 May 09:  <a href="ap200509.html">Full Flower Moonrise</a><br/>
 2020 May 08:  <a href="ap200508.html">Long Tailed Comet SWAN</a><br/>
 2020 May 07:  <a href="ap200507.html">Analemma of the Moon</a><br/>
 2020 May 06:  <a href="ap200506.html">LDN 1471: A Windblown Star Cavity</a><br/>
 2020 May 05:  <a href="ap200505.html">Carina in Perspective</a><br/>
 2020 May 04:  <a href="ap200504.html">Earth Flyby of 

In [23]:
#Limpieza
#Obtiene el url de cada fecha la primera url es de la fecha mas reciente
links=[] 
for link in soup.find_all('a', href=True):
    if '.html' in str(link):
        if 'ap' in str(link):
            #añade el primer dominio para crear la url completa
            links.append('https://apod.nasa.gov/apod/'+link['href'])

#Hay 2 links extras en nuestro df se eliminan

links.remove('https://apod.nasa.gov/apod/lib/aptree.html')

#'https://apod.nasa.gov/apod/lib/about_apod.html' este es el ultimo link usamos pop para que se elimine del dataframe
links.pop(-1)


#Volteamos la lista para empezar con 2015-01-01 que es la primera foto de APOD

links_orden=links[::-1]

#data frame de links
df_links=pd.DataFrame(links_orden)
df_links

,0
0,https://apod.nasa.gov/apod/ap150101.html
1,https://apod.nasa.gov/apod/ap150102.html
2,https://apod.nasa.gov/apod/ap150103.html
3,https://apod.nasa.gov/apod/ap150104.html
4,https://apod.nasa.gov/apod/ap150105.html
...,...
1955,https://apod.nasa.gov/apod/ap200509.html
1956,https://apod.nasa.gov/apod/ap200510.html
1957,https://apod.nasa.gov/apod/ap200511.html
1958,https://apod.nasa.gov/apod/ap200512.html


In [24]:
#lista que obtiene solo texto de la sopa, dando resultado listas de listas.

fecha_titulo_paso_1=[b.text.replace('\n\n','\n').strip().split("\n") for b in archive_extract]

#La posicion 1 de la lista fecha_titulo1 es la que contiene la lista de fechas y titulos.
fecha_titulo_paso_2=fecha_titulo_paso_1[1]

# El texto se separara por fechas y por titulo

fecha_titulo=[texto.split(':') for texto in fecha_titulo_paso_2]



#Volteamos la lista para empezar con 2015-01-01 que es la primera foto de APOD

fecha_titulo_orden=fecha_titulo[::-1]

#Pre data frame, al realizar el dataframe nos damos cuenta que hay una columna con valores:None , se eliminara.
df_fecha_titulo=pd.DataFrame(fecha_titulo_orden)

del(df_fecha_titulo[2])

#Concatenaremos los dos df:

# df3= pd.concat([df1, df3], axis=1,)

df_archive=pd.concat([df_fecha_titulo,df_links],axis=1)
df_archive

,0,1,0
0,2015 January 01,Vela Supernova Remnant,https://apod.nasa.gov/apod/ap150101.html
1,2015 January 02,At the Heart of Orion,https://apod.nasa.gov/apod/ap150102.html
2,2015 January 03,Apollo 17,https://apod.nasa.gov/apod/ap150103.html
3,2015 January 04,Crescent Rhea Occults Crescent Saturn,https://apod.nasa.gov/apod/ap150104.html
4,2015 January 05,"A Fox Fur, a Unicorn, and a Christmas Tree",https://apod.nasa.gov/apod/ap150105.html
...,...,...,...
1955,2020 May 09,Full Flower Moonrise,https://apod.nasa.gov/apod/ap200509.html
1956,2020 May 10,The Porpoise Galaxy from Hubble,https://apod.nasa.gov/apod/ap200510.html
1957,2020 May 11,Behind Betelgeuse,https://apod.nasa.gov/apod/ap200511.html
1958,2020 May 12,Lyrid Meteors from the Constellation Lyra,https://apod.nasa.gov/apod/ap200512.html


In [28]:
#Para continuar con correr API hay que saber cuantos request seran necesarios, para eso : hay que saber la longitud de filas en fecha titulo

longitud_web_scrapping= len(fecha_titulo)
print(longitud_web_scrapping)

1960


In [27]:
#Data_frame_to_csv
df_archive.to_csv('Web_scraping_nasa_apod_urls_.csv')